# Kernel Memory plugin for Semantic Kernel

This notebook shows the basic usage of Kernel Memory as a Semantic Kernel Plugin.

First of all, install the Kernel Memory SK Plugin and SK dependencies.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.20.231211.3"

using Microsoft.SemanticKernel;
using Microsoft.KernelMemory;

Installed Packages Microsoft.KernelMemory.SemanticKernelPlugin, 0.20.231211.3 Microsoft.SemanticKernel, 1.0.0-rc3

## Configuration

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with your OpenAI API Key, and your Memory Service API Key (if you set one).

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> MEMORY_API_KEY=<your KM web service API key>
> ```

In [2]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

Let's setup Semantic Kernel as usual:

In [3]:
var kernel = new KernelBuilder()
    .AddOpenAIChatCompletion(
        modelId: "gpt-3.5-turbo",
        apiKey: env["OPENAI_API_KEY"])
    .Build();

Console.WriteLine("Semantic Kernel ready.");

Semantic Kernel ready.


## Import Memory Plugin into SK

Let's load Kernel Memory plugin into SK.

Remember to start the memory service on localhost, otherwise change the URL and
point it to your Kernel Memory service endpoint. In that case you should also
provide the API key protecting your KM deployment.

In [4]:
// This is the endpoint where you're running the memory service
// Note: the TCP port might be different, depending on how you run it.
var memoryServiceEndpoint = "http://127.0.0.1:9001/";

// and the API Key you've configured. env["MEMORY_API_KEY"] contains the value from the .env file
var memoryServiceAPIKey = env["MEMORY_API_KEY"];

// Instance of the HTTP client used to talk to KM service
var memoryConnector = new MemoryWebClient(memoryServiceEndpoint, memoryServiceAPIKey);

// Name of the plugin. This is the name you'll use in skPrompt, e.g. {{memory.ask ...}}
var pluginName = "memory";

// Import the plugin into the kernel.
// 'waitForIngestionToComplete' set to true forces memory write operations to wait for completion.
var memoryPlugin = kernel.ImportPluginFromObject(
    new MemoryPlugin(memoryConnector, waitForIngestionToComplete: true),
    pluginName);

Console.WriteLine("Memory plugin imported.");

Memory plugin imported.


## Populate memory with sample information

Let's store some data in memory:

1. storing some text, e.g. Wikipedia's description of Orion
2. and importing a PDF document, e.g. a news from NASA about Orion spacecraft

To import data you can use either the web client connector or the plugin, here we use both just for demo purposes.

When working with SK Planners and Semantic Functions (like the one below) you should always
use KM Plugin. You can also use the web client connector to create new plugins and customize
how you interact with your memories, e.g. selecting indexes, using filters, customizing chunking, etc.

In [5]:
// Save a string using the web client
await memoryConnector.ImportTextAsync(
    "Orion is a prominent set of stars visible during winter in " +
    "the northern celestial hemisphere. It is one of the 88 modern constellations; " +
    "it was among the 48 constellations listed by the 2nd-century astronomer Ptolemy. " +
    "It is named for a hunter in Greek mythology.", 
    documentId: "OrionDefinition");

// Save a PDF file using the plugin
var context = new KernelArguments
{
    [MemoryPlugin.FilePathParam] = "NASA-news.pdf",
    [MemoryPlugin.DocumentIdParam] = "NASA001"
};
await memoryPlugin["SaveFile"].InvokeAsync(kernel, context);

Console.WriteLine("Memory updated.");

Memory updated.


## Use Memory in a semantic function

Here we define a simple semantic function, using `{{memory.ask}}` to
fetch information from memory.

In [6]:
var skPrompt = """
Question to Memory: {{$input}}

Answer from Memory: {{memory.ask $input}}

If the answer is empty say 'I don't know' otherwise reply with a preview of the answer,
truncated to 15 words. Prefix with one emoji relevant to the content.
""";

var myFunction = kernel.CreateFunctionFromPrompt(skPrompt);

Console.WriteLine("Semantic Function ready.");

Semantic Function ready.


Let's now interact with our memories using `myFunction` function, asking questions, leveraging KM and LLMs to generate answers.

Our function internally uses KM Plugin to retrieve an answer, and then custom some semantic logic to decide how to format the output.

In [7]:
var answer = await myFunction.InvokeAsync(kernel,
    "any news from NASA about Orion?");

Console.WriteLine(answer);

🚀 NASA has invited media to see the new test version of the Orion spacecraft and recovery hardware.


In [8]:
var answer = await myFunction.InvokeAsync(kernel,
    "define Orion");

Console.WriteLine(answer);

🌌 Orion is a constellation named after a Greek hunter. It is visible in winter and has a spacecraft named after it.
